# Задание
Выполните следующие задания:

> 1. Лёгкий уровень (Задание не обязательно к выполнению)
Подставьте в нейронные сети из п.3.4.3 (шпаргалки) по распознаванию тональностей новостей слой LSTM и обучите нейронные сети. Сделайте выводы, улучшила ли подстановка слоя качество распознавания тональностей.


> 2. PRO уровень (Задание не обязательно к выполнению)
Найдите код модели классификации BERT. Возьмите русскую модель, доучите её и запустите классификацию тональностей отзывов на данной модели. Можно не добиваться высоких показателей метрик — примеров для обучения мало для существенного роста метрик модели.

# 1

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from pprint import pprint
import json
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import LSTM, RepeatVector
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Embedding, SpatialDropout1D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy

In [ ]:
PPRINT_WIDTH = 160 # константа для функции pprint, количество символов в одной строке при выводе

In [ ]:
def acc_loss(model):
    acc = model.history['accuracy']
    val_acc = model.history['val_accuracy']

    loss = model.history['loss']
    val_loss = model.history['val_loss']

    # построение графика точности
    plt.plot(acc, 'r', label='Train Accuracy')
    plt.plot(val_acc, 'b', label='Validation Accuracy')
    plt.title('Accuracy vs. Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    # построение графика ошибки
    plt.plot(loss, 'r', label='Train Loss')
    plt.plot(val_loss, 'b', label='Validation Loss')
    plt.title('Loss vs. Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
path_train = './train.json'

text_data = pd.read_json(path_train)
text_data.drop(columns = 'id', inplace = True)
print(text_data.shape)
text_data.head()

In [ ]:
text_data.shape # выведем размерность нашего DataFrame

In [ ]:
#  сделаем разделение наших данных на обучение тест с учетом стратификации
train_index, test_index = train_test_split(np.arange(text_data.shape[0]), stratify = text_data['sentiment'])

x_train_raw = text_data.iloc[train_index, 0].values
y_train_raw = text_data.iloc[train_index, 1].values
x_test_raw = text_data.iloc[test_index, 0].values
y_test_raw = text_data.iloc[test_index, 1].values

In [ ]:
NUM_WORDS = 20000 # константа, максимальное количество слов, которые будет учитывать наша модель

# создадим наш Токенайзер
tokenizer = Tokenizer(num_words=NUM_WORDS, # максимальное количество слов, которые будет учитывать наш токенайзер
                      filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', # символы, которые он токенайзер будет фильтровать
                      lower=True,  #приводятся ли все символы к нижнему регситру
                      split=' ',  # символ, по которому происходит разделение на слова (токены)
                      char_level=False,  # являются ли токенами отдельные буквы
                      oov_token='UNKNOWN' # токен для неизвестных слов
                     )

tokenizer.fit_on_texts(x_train_raw) # обучим наш токенайзер на обучающих текстах

In [ ]:
x_train_seq = tokenizer.texts_to_sequences(x_train_raw) # преобразуем наши тексты в последовательность токенов (индексов слов)

In [ ]:
# а теперь попробуем сделать обратное декодирование нашей последовательности в текст
# это позволит нам понять, какие данные видит нейронная сеть
text = ' '
for i in x_train_seq[1]:
    text += tokenizer.index_word[i] + ' '
pprint(text, width=PPRINT_WIDTH)

In [ ]:
# Преобразуем наши текстовые данные в формат One Hot Encoding
x_train_01 = tokenizer.texts_to_matrix(x_train_raw)
x_test_01 = tokenizer.texts_to_matrix(x_test_raw)

In [ ]:
# закодируем нашу целевую переменную (класс отзыва) в формат OHE для подачи в нейронную сеть
target_encoeder = OneHotEncoder(sparse=False) # создадим объект Encoder
target_encoeder.fit(y_train_raw.reshape([-1, 1])) # обучим его на целевом признаке из обучающих данных

In [ ]:
# сохраним названия классов в отдельную переменную, это понадобится нам на этапе предсказани
classes_names = target_encoeder.categories_[0] # названия классов хранятся в .categories_[0]
classes_names

In [ ]:
# приведем наши целевые переменные из называний в формат OHO для подачи в нейронную сеть
y_train_01 = target_encoeder.transform(y_train_raw.reshape([-1, 1]))
y_test_01 = target_encoeder.transform(y_test_raw.reshape([-1, 1]))

In [ ]:
# подсчитаем количество уникальных классов отзывов
n_classes = text_data['sentiment'].nunique()

In [ ]:
# переводим наши тексты в последовательность индексов (токенов) с помощью tokenizer
x_train_seq = tokenizer.texts_to_sequences(x_train_raw)
x_test_seq = tokenizer.texts_to_sequences(x_test_raw)

In [ ]:
# объявим функцию для чистки наших последовательностей от тега unknown
# мы предполагаем, что наличие тега unknown не несет значимой информации
def drop_UNKNOWN (x_seq, unknown=1):
    x_seq_short = []
    for x in x_seq:
        x_ = np.array(x)
        x_ = x_[np.where(x_ !=unknown )]
        x_seq_short.append(list(x_))
    return x_seq_short

In [ ]:
# устанавливаем максимальную длинну последовательности токенов
MAX_LEN_SEQ = 2000

# очистим наши последовательности, полученные из обучающей и тестовой выборок
# от тега unknown с использованием объявленной функции
x_train_seq_short = drop_UNKNOWN(x_train_seq)
x_test_seq_short = drop_UNKNOWN(x_test_seq)

In [ ]:
# вырвниваем длинну всех последовательностей токенов до MAX_LEN_SEQ
# при помощи стандартного инструмента pad_sequence, входящего в Keras
# при этом последовательности короче MAX_LEN_SEQ будут дополнены нулями
# а последовательности длиннее MAX_LEN_SEQ будут обрезаны

x_train_emb = pad_sequences(x_train_seq_short, padding='post', maxlen=MAX_LEN_SEQ)
x_test_emb = pad_sequences(x_test_seq_short, padding='post', maxlen=MAX_LEN_SEQ)

In [ ]:
# объявим нашу модель
modelEmb = Sequential() # объявляем нашу модель как последовательность слоев
# добавляем слой Embedding
modelEmb.add(Embedding(input_dim=NUM_WORDS, output_dim=200, input_length=MAX_LEN_SEQ))
# добавляем слой SpatialDropout1D для "прореживания" и борьбы с переобучением
modelEmb.add(SpatialDropout1D(0.5))
# добавим выравнивающий слой
modelEmb.add(Flatten())
# добавим Dense слой на 16 нейронов
modelEmb.add(Dense(16,  activation='relu'))
# добавим батч-нормализацию для борьбы с переобучением
modelEmb.add(BatchNormalization())
# добавим прореживание для борьбы с переобучением
modelEmb.add(Dropout(0.2))
# добавим выходной полносвязный слой для классификации
modelEmb.add(Dense(n_classes, activation='softmax'))

# компилируем модель
modelEmb.compile(optimizer=Adam(learning_rate=0.001),  loss='categorical_crossentropy',  metrics=['accuracy'])

# выводим данные по модели
modelEmb.summary()

In [ ]:
model = modelEmb.fit(x = x_train_emb,  y = y_train_01, epochs = 10, verbose = 1, validation_data= (x_test_emb, y_test_01))

In [ ]:
acc_loss(model)

### LSTM

In [ ]:
# объявим нашу модель LSTM
# Создание модели
model_LSTM = Sequential() # объявляем нашу модель как последовательность слоев
# добавляем слой Embedding
model_LSTM.add(Embedding(input_dim=NUM_WORDS, output_dim=200, input_length=MAX_LEN_SEQ))

# добавляем слой SpatialDropout1D для "прореживания" и борьбы с переобучением
model_LSTM.add(SpatialDropout1D(0.5))

# Добавляем слой долго-краткосрочной памяти (400 элементов для долговременного хранения информации, отключаем входной сигнал с вероятностью 20%, отключаем рекуррентный сигнал с вероятностью 20%)
model_LSTM.add(LSTM(400, dropout=0.2, recurrent_dropout=0.2))

# добавим выравнивающий слой
model_LSTM.add(Flatten())

# добавим Dense слой на 96 нейронов
model_LSTM.add(Dense(96, activation='relu'))

# добавим Dense слой на 16 нейронов
model_LSTM.add(Dense(16, activation='relu'))

# добавим выходной полносвязный слой для классификации
model_LSTM.add(Dense(n_classes,activation='softmax'))
model_LSTM.add(Dropout(0.2))

# компилируем модель
model_LSTM.compile(optimizer=Adam(learning_rate=0.001),  loss='categorical_crossentropy',  metrics=['accuracy'])

In [ ]:
model_LSTM.summary()

In [ ]:
model = model_LSTM.fit(x=x_train_emb, y=y_train_01, epochs=10, verbose=1, validation_data=(x_test_emb, y_test_01))

In [ ]:
acc_loss(model)